# Homework 2 - Applied Genomics Sec 3 - Alignment

#### In this assignment, the focus is on learning how to produce a well-annotated Jupyter Notebook, and using your experience with running bwa to run a different aligner.

#### Textboxes like this one can be produced by selecting "Markdown" instead of "Code" in the menu at the top. 

You can manipulate the size of the text by adding different numbers of "#" at the beginning of the text. Double-click on any of the other textboxes to see.

The sequence data we will be aligning is from an RNAseq [study](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0217309) which looks at differential gene expression in roots after inoculation with growth-promoting bacteria. (Double-click on this textbox to see the link to the study if the hyperlink isn't visible)

## Section 1: Setting up (5x4 = 20 points)

#### This will count as a brief review of basic bash commands

#### 1. Copy the homework_2 folder /scratch/work/courses/AppliedGenomicsSec3/students/homework_2/ into your scratch directory.

In [2]:
cp -r /scratch/work/courses/AppliedGenomicsSec3/students/homework_2/ .

#### 2. Change into the Homework2 folder in your scratch directory and observe what's inside the folder along with the size of the files in human-readable format

In [3]:
cd homework_2

In [4]:
ls -l

total 25093249
-rw-r--r--. 1 sg5096 sg5096   390983951 Feb 21 17:01 OSativaR498.fa
-rw-r--r--. 1 sg5096 sg5096 12652199556 Feb 21 17:00 SRR8695538_1.fastq
-rw-r--r--. 1 sg5096 sg5096 12652199556 Feb 21 17:01 SRR8695538_2.fastq
-rw-r--r--. 1 sg5096 sg5096         203 Feb 21 15:34 process_sam.sh
-rw-r--r--. 1 sg5096 sg5096         275 Feb 21 17:00 run_hisat2.sh


#### 3. Run a command to see how many reads SRR8695538_1.fastq contains

In [5]:
cat SRR8695538_1.fastq | grep '@SRR*' | wc -l

33064177


#### 4. Look for and load the module HiSAT2

In [6]:
module avail HiSAT2


--------------------------- /share/apps/modulefiles ----------------------------
   hisat2/2.2.1


In [7]:
module load hisat2/2.2.1

#### 5. Print out and look at the usage documentation for the commands hisat2 and hisat2-build

In [22]:
hisat2 

No index, query, or output file specified!
HISAT2 version 2.2.1 by Daehwan Kim (infphilo@gmail.com, www.ccb.jhu.edu/people/infphilo)
Usage: 
  hisat2 [options]* -x <ht2-idx> {-1 <m1> -2 <m2> | -U <r> | --sra-acc <SRA accession number>} [-S <sam>]

  <ht2-idx>  Index filename prefix (minus trailing .X.ht2).
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <SRA accession number>        Comma-separated list of SRA accession numbers, e.g. --sra-acc SRR353653,SRR353654.
  <sam>      File for SAM output (default: stdout)

  <m1>, <m2>, <r> can be comma-separated lists (no whitespace) and can be
  specified many times.  E.g. '-U file1.fq,file2.fq -U 

: 1

In [13]:
hisat2-build 

No input sequence or sequence file specified!
HISAT2 version 2.2.1 by Daehwan Kim (infphilo@gmail.com, http://www.ccb.jhu.edu/people/infphilo)
Usage: hisat2-build [options]* <reference_in> <ht2_index_base>
    reference_in            comma-separated list of files with ref sequences
    hisat2_index_base       write ht2 data to files with this dir/basename
Options:
    -c                      reference sequences given on cmd line (as
                            <reference_in>)
    --large-index           force generated index to be 'large', even if ref
                            has fewer than 4 billion nucleotides
    -a/--noauto             disable automatic -p/--bmax/--dcv memory-fitting
    -p <int>                number of threads
    --bmax <int>            max bucket sz for blockwise suffix-array builder
    --bmaxdivn <int>        max bucket sz as divisor of ref len (default: 4)
    --dcv <int>             diff-cover period for blockwise (default: 1024)
    --nodc              

: 1

## Section 2: Using HISAT to align sequences to reference (40 points)

#### 1. Write an sbatch script called run_hisat2.sh to run alignment on the paired fastq files. Remember to provide the code you used to index the reference fasta within this script
#### In the sbatch script, set cpus-per-task=6 and mem=10GB
#### Ask for 6 threads while running the hisat2 command using the --threads argument
#### Print out the contents of the sbatch script here (20 points)

In [8]:
cat run_hisat2.sh

#!/bin/bash
#SBATCH --cpus-per-task=6
#SBATCH --time=5:00:00
#SBATCH --mem=10GB
#SBATCH --job-name=hisat


module purge
module load hisat2/2.2.1

index = $1
LREAD = $2
RREAD = $3

hisat2-build -f $index.fa $index

hisat2 --threads=6 -x $index -1 $LREAD -2 $RREAD > $LREAD.sam

#### 2. Run the job and once it's complete, check the memory expenditure (5 points)

*seff* is a command that will tell you how much resources your slurm job needed to run.

To use the command simply type seff followed by the slurm job id. Remember the slurm job id is also saved as part of the slurm job output file.

In [10]:
sbatch run_hisat2.sh OSativaR498 SRR8695538_1.fastq SRR8695538_2.fastq

Submitted batch job 15126308


In [47]:
seff 15126308

Job ID: 15126308
Cluster: greene
User/Group: sg5096/sg5096
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 6
CPU Utilized: 01:03:01
CPU Efficiency: 62.02% of 01:41:36 core-walltime
Job Wall-clock time: 00:16:56
Memory Utilized: 739.17 MB
Memory Efficiency: 7.22% of 10.00 GB


#### 3. Would bwa have worked just as well for RNA-sequence data? Why or why not? (10 points)

## Section 3: Introducing SAMtools (45 points)

#### Load the SAMtools module and explore the usage documentation for the commands view, index, sort, and flagstat by printing out the help documentation for each. 
#### Add as many code boxes as you need (5 points)

In [16]:
module avail SAMtools


--------------------------- /share/apps/modulefiles ----------------------------
   samtools/intel/1.11    samtools/intel/1.12    samtools/intel/1.14


In [18]:
module load samtools/intel/1.11  

In [19]:
samtools


Program: samtools (Tools for alignments in the SAM format)
Version: 1.11 (using htslib 1.11)

Usage:   samtools <command> [options]

Commands:
  -- Indexing
     dict           create a sequence dictionary file
     faidx          index/extract FASTA
     fqidx          index/extract FASTQ
     index          index alignment

  -- Editing
     calmd          recalculate MD/NM tags and '=' bases
     fixmate        fix mate information
     reheader       replace BAM header
     targetcut      cut fosmid regions (for fosmid pool only)
     addreplacerg   adds or replaces RG tags
     markdup        mark duplicates
     ampliconclip   clip oligos from the end of reads

  -- File operations
     collate        shuffle and group alignments by name
     cat            concatenate BAMs
     merge          merge sorted alignments
     mpileup        multi-way pileup
     sort           sort alignment file
     split          splits a file by read group
     quickcheck     quickly check if SA

: 1

In [36]:
samtools index

Usage: samtools index [-bc] [-m INT] <in.bam> [out.index]
Options:
  -b       Generate BAI-format index for BAM files [default]
  -c       Generate CSI-format index for BAM files
  -m INT   Set minimum interval size for CSI indices to 2^INT [14]
  -@ INT   Sets the number of threads [none]


: 1

In [37]:
samtools view


Usage: samtools view [options] <in.bam>|<in.sam>|<in.cram> [region ...]

Options:
  -b       output BAM
  -C       output CRAM (requires -T)
  -1       use fast BAM compression (implies -b)
  -u       uncompressed BAM output (implies -b)
  -h       include header in SAM output
  -H       print SAM header only (no alignments)
  -c       print only the count of matching records
  -o FILE  output file name [stdout]
  -U FILE  output reads not selected by filters to FILE [null]
  -t FILE  FILE listing reference names and lengths (see long help) [null]
  -X       include customized index file
  -L FILE  only include reads overlapping this BED FILE [null]
  -r STR   only include reads in read group STR [null]
  -R FILE  only include reads with read group listed in FILE [null]
  -d STR:STR
           only include reads with tag STR and associated value STR [null]
  -D STR:FILE
           only include reads with tag STR and associated values listed in
           FILE [null]
  -q INT   only in

In [38]:
samtools sort

Usage: samtools sort [options...] [in.bam]
Options:
  -l INT     Set compression level, from 0 (uncompressed) to 9 (best)
  -u         Output uncompressed data (equivalent to -l 0)
  -m INT     Set maximum memory per thread; suffix K/M/G recognized [768M]
  -M         Use minimiser for clustering unaligned/unplaced reads
  -K INT     Kmer size to use for minimiser [20]
  -n         Sort by read name (not compatible with samtools index command)
  -t TAG     Sort by value of TAG. Uses position as secondary index (or read name if -n is set)
  -o FILE    Write final output to FILE rather than standard output
  -T PREFIX  Write temporary files to PREFIX.nnnn.bam
  --no-PG    do not add a PG line
      --input-fmt-option OPT[=VAL]
               Specify a single input file format option in the form
               of OPTION or OPTION=VALUE
  -O, --output-fmt FORMAT[,OPT[=VAL]]...
               Specify output format (SAM, BAM, CRAM)
      --output-fmt-option OPT[=VAL]
               Specify a

In [39]:
samtools flagstat

Usage: samtools flagstat [options] <in.bam>
      --input-fmt-option OPT[=VAL]
               Specify a single input file format option in the form
               of OPTION or OPTION=VALUE
  -@, --threads INT
               Number of additional threads to use [0]
      --verbosity INT
               Set level of verbosity
  -O, --output-fmt FORMAT[,OPT[=VAL]]...
               Specify output format (json, tsv)


#### SAM files are often converted to the smaller, binary BAM format for more efficient storage of the same information. 
#### Use samtools to convert your output SAM file from the previous section into a BAM file.
#### This is going to take a few minutes, so do the conversion via an sbatch script called process_sam.sh. Ask for 4GB of memory
#### Print out the contents of the script, submit the job, and print out the sizes of the SAM and BAM files after the job is completed (10 points)

In [16]:
cat process_sam.sh

#!/bin/bash
#SBATCH --cpus-per-task=6
#SBATCH --time=5:00:00
#SBATCH --mem=4GB
#SBATCH --job-name=samtools


module purge
module load samtools/intel/1.11 

Name=$1

samtools view -b $Name.sam > $Name.bam

In [21]:
sbatch process_sam.sh SRR8695538_1.fastq

Submitted batch job 15128909


In [52]:
ls -l SRR8695538_1.*

-rw-r--r--. 1 sg5096 sg5096 12652199556 Feb 21 17:00 SRR8695538_1.fastq
-rw-rw-r--. 1 sg5096 sg5096  6312041091 Feb 21 17:46 SRR8695538_1.fastq.bam
-rw-rw-r--. 1 sg5096 sg5096 27922090096 Feb 21 17:23 SRR8695538_1.fastq.sam
-rw-rw-r--. 1 sg5096 sg5096  4477060508 Feb 21 18:16 SRR8695538_1.fastq.sorted.bam


In [43]:
seff 15128909

Job ID: 15128909
Cluster: greene
User/Group: sg5096/sg5096
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 6
CPU Utilized: 00:17:02
CPU Efficiency: 16.57% of 01:42:48 core-walltime
Job Wall-clock time: 00:17:08
Memory Utilized: 18.68 MB
Memory Efficiency: 0.46% of 4.00 GB


#### We will continue to work with the process_sam.sh. To prevent the code from repeating the sam to bam conversion, comment out the line converting SAM to BAM using "#". It should read something like:
#### #samtools view ...
#### Add a new line of code to sort the BAM file using samtools. Submit the job and explain why sorting a BAM file is useful in the second box. (10 points)

In [45]:
sbatch process_sam.sh SRR8695538_1.fastq

Submitted batch job 15130234


In [51]:
seff 15130234

Job ID: 15130234
Cluster: greene
User/Group: sg5096/sg5096
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 6
CPU Utilized: 00:20:28
CPU Efficiency: 16.41% of 02:04:42 core-walltime
Job Wall-clock time: 00:20:47
Memory Utilized: 857.66 MB
Memory Efficiency: 20.94% of 4.00 GB


 Sorting a BAM file is useful because it sorts the file in genome order. This means the alignments are sorted positionally, based on the alignment coordinate on each chromosome. Without sorting, it is in random order, which makes any analysis difficult. 

#### Similar to above, comment out the line sorting the BAM file, and add a new line of code to index the sorted BAM file using samtools. Submit the job and explain why indexing a BAM file is useful in the second box. (10 points)

In [54]:
sbatch process_sam.sh SRR8695538_1.fastq

Submitted batch job 15130560


In [56]:
seff 15130560

Job ID: 15130560
Cluster: greene
User/Group: sg5096/sg5096
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 6
CPU Utilized: 00:01:16
CPU Efficiency: 15.64% of 00:08:06 core-walltime
Job Wall-clock time: 00:01:21
Memory Utilized: 12.96 MB
Memory Efficiency: 0.32% of 4.00 GB


Indexing allows for processes working on the BAM files to be completed faster, because BAM files are large and if not indexed would take longer to process. 

#### Print out the final contents of process_sam.sh using the cat command

#### Run flagstat (no need to run this as an sbatch job) to find out the percentage of primary mapped reads. What do secondary mapped reads represent? (10 points

In [57]:
cat process_sam.sh

#!/bin/bash
#SBATCH --cpus-per-task=6
#SBATCH --time=5:00:00
#SBATCH --mem=4GB
#SBATCH --job-name=samtools


module purge
module load samtools/intel/1.11 

Name=$1

#samtools view -b $Name.sam > $Name.bam
#samtools sort $Name.bam -o $Name.sorted.bam
samtools index $Name.sorted.bam

In [59]:
samtools flagstat SRR8695538_1.fastq.bam

69192564 + 0 in total (QC-passed reads + QC-failed reads)
3064210 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
39171960 + 0 mapped (56.61% : N/A)
66128354 + 0 paired in sequencing
33064177 + 0 read1
33064177 + 0 read2
30730204 + 0 properly paired (46.47% : N/A)
34530496 + 0 with itself and mate mapped
1577254 + 0 singletons (2.39% : N/A)
45946 + 0 with mate mapped to a different chr
31911 + 0 with mate mapped to a different chr (mapQ>=5)


Based on the above results from using flagstat, the percentage of primary mapped reads is 0%. Secondary mapped reads represent

## Export this notebook as an html file (File -> Export Notebook As... -> HTML) and turn in the html file on Brightspace